In [ ]:
!pip install plotly

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
data = [
    ['ozon.ru/product/293783766', 0.8, 2776, 4.8, 68],
    ['ozon.ru/product/151517600', 0.65, 6394, 4.7, 80],
    ['ozon.ru/product/8751202', 2.4, 4468, 4.9, 84],
    ['ozon.ru/product/147927590', 1.5, 8090, 4.85, 76],
    ['ozon.ru/product/278676526', 1.2, 2490, 4.8, 75],
    ['ozon.ru/product/185555856', 0.6, 12143, 4.8, 69],
    ['ozon.ru/product/146189876', 1.3, 5257, 4.75, 83],
    ['ozon.ru/product/143167442', 2, 7590, 4.9, 79],
    ['ozon.ru/product/141378152', 2.5, 5690, 4.9, 83],
    ['ozon.ru/product/29636513', 4, 12790, 4.9, 78]
]

df = pd.DataFrame(data, columns=['link', 'volume', 'price', 'rating', 'noise'])

In [ ]:
fig = px.scatter(df, x='volume_rank', y='price_rank')
fig.show()

In [ ]:
df = df.sort_values(['volume_rank', 'price_rank'])

In [ ]:
df

In [ ]:
from functools import reduce

In [ ]:
def mult(el_prev, el):
    return el_prev*el

In [ ]:
is_dominated({'volume_rank': 5,
              'price_rank': 5}, df)

In [ ]:
params = {x[0]: {'volume_rank': x[6],
                 'price_rank': x[7]} for x in df.to_records()}

In [ ]:
df

In [ ]:
params

In [ ]:
pareto_1 = [[key, is_dominated(params[key], df)] for key in params.keys()]

In [ ]:
pareto_1_df = df.join(pd.DataFrame(pareto_1, index=[x[0] for x in pareto_1],
                                   columns=['index', 'is_dominated'])).drop(columns=['index'])

In [ ]:
pareto_1_df

In [ ]:
fig = px.scatter(pareto_1_df, x='volume_rank', y='price_rank', color = 'is_dominated')
fig.show()

In [ ]:
params_1 = {x[0]: {'volume_rank': x[4],
                   'price_rank': x[5]} for x in pareto_1_df[pareto_1_df['is_dominated']].to_records()}

In [ ]:
pareto_2 = [[key, is_dominated(params[key], pareto_1_df[pareto_1_df['is_dominated']])] for key in params.keys()]

In [ ]:
pareto_2_df = pareto_1_df.join(pd.DataFrame(pareto_2, index=[x[0] for x in pareto_2],
                                   columns=['index', 'is_dominated_2'])).drop(columns=['index'])

In [ ]:
pareto_2_df['pareto_set'] = pareto_2_df['is_dominated'].astype(int) + pareto_2_df['is_dominated_2'].astype(int)
pareto_2_df['pareto_set'] = pareto_2_df['pareto_set'].astype(str)
pareto_2_df['size'] = 1.5

In [ ]:
fig = px.scatter(pareto_2_df, x='volume_rank', y='price_rank', color = 'pareto_set', size='size')
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

In [ ]:
pareto_2_df['ip'] = (10 - pareto_2_df['volume_rank'])**2 + (10 - pareto_2_df['price_rank'])**2

In [ ]:
pareto_2_df[pareto_2_df['pareto_set']=='0'].sort_values('ip')

In [ ]:
pareto_2_df[pareto_2_df['pareto_set']=='1'].sort_values('ip')

In [ ]:
df

In [3]:
import numpy as np

In [4]:
def is_dominated(list1, list2):
    if len(list1) == len(list2):
        compared = []
        for i in range(len(list1)):
            compared.append(list1[i]>list2[i])
        return np.prod(compared)

In [5]:
def dominates(series):
    bool_list = []
    for record in series:
        dominators = []
        for pos_d in series:
            if np.prod(np.array(pos_d)>np.array(record)) == 1:
                dominators.append(pos_d)
        if len(dominators) > 0:
            bool_list.append(True)
        else:
            bool_list.append(False)
    return bool_list

In [6]:
def pareto_set(df_raw, param_dict):
    df_proc = df_raw.copy()[list(param_dict.keys())]
    for param in param_dict.keys():
        df_proc[param+'_rank'] = df_proc[param].rank(ascending=param_dict[param], method='dense')
    ranks = [param+'_rank' for param in param_dict.keys()]
    df_proc['is_dominated'] = dominates(df_proc[ranks].values.tolist())
    return df_proc

In [9]:
pareto_1 = pareto_set(df, {'volume': True,
                           'price': False})

In [10]:
pareto_1

,volume,price,volume_rank,price_rank,is_dominated
0,0.80,2776,3.0,9.0,True
1,0.65,6394,2.0,5.0,True
2,2.40,4468,8.0,8.0,False
3,1.50,8090,6.0,3.0,True
4,1.20,2490,4.0,10.0,False
5,0.60,12143,1.0,2.0,True
6,1.30,5257,5.0,7.0,True
7,2.00,7590,7.0,4.0,True
8,2.50,5690,9.0,6.0,False
9,4.00,12790,10.0,1.0,False


In [ ]:
pareto_set(df, {'volume': True,
                'price': False})

In [14]:
pareto_2 = pareto_set(pareto_1[pareto_1['is_dominated']], {'volume': True,
                                                           'price': False})

In [15]:
pareto_2

,volume,price,volume_rank,price_rank,is_dominated
0,0.80,2776,3.0,6.0,False
1,0.65,6394,2.0,4.0,True
3,1.50,8090,5.0,2.0,True
5,0.60,12143,1.0,1.0,True
6,1.30,5257,4.0,5.0,False
7,2.00,7590,6.0,3.0,False


In [17]:
pareto_12 = pareto_1.join(pareto_2['is_dominated'], how='left', lsuffix='2')

In [22]:
pareto_12['group'] = pareto_12['is_dominated'].fillna(False).astype(int)+pareto_12['is_dominated2'].fillna(False).astype(int)

In [28]:
pareto_12['group'] = pareto_12['group'].astype(str)
pareto_12['size'] = 2

In [51]:
pareto_12

,volume,price,volume_rank,price_rank,is_dominated2,is_dominated,group,size
0,0.80,2776,3.0,9.0,True,False,1,2
1,0.65,6394,2.0,5.0,True,True,2,2
2,2.40,4468,8.0,8.0,False,NaN,0,2
3,1.50,8090,6.0,3.0,True,True,2,2
4,1.20,2490,4.0,10.0,False,NaN,0,2
5,0.60,12143,1.0,2.0,True,True,2,2
6,1.30,5257,5.0,7.0,True,False,1,2
7,2.00,7590,7.0,4.0,True,False,1,2
8,2.50,5690,9.0,6.0,False,NaN,0,2
9,4.00,12790,10.0,1.0,False,NaN,0,2


In [29]:
fig = px.scatter(pareto_12, x='volume_rank', y='price_rank', color = 'group', size = 'size')
fig.show()

In [30]:
df.head()

,link,volume,price,rating,noise
0,ozon.ru/product/293783766,0.80,2776,4.80,68
1,ozon.ru/product/151517600,0.65,6394,4.70,80
2,ozon.ru/product/8751202,2.40,4468,4.90,84
3,ozon.ru/product/147927590,1.50,8090,4.85,76
4,ozon.ru/product/278676526,1.20,2490,4.80,75


In [31]:
pareto_1_2 = pareto_set(df, {'volume': True,
                             'price': False,
                             'rating': True,
                             'noise': False})

In [32]:
pareto_1_2

,volume,price,rating,noise,volume_rank,price_rank,rating_rank,noise_rank,is_dominated
0,0.80,2776,4.80,68,3.0,9.0,3.0,9.0,False
1,0.65,6394,4.70,80,2.0,5.0,1.0,3.0,True
2,2.40,4468,4.90,84,8.0,8.0,5.0,1.0,False
3,1.50,8090,4.85,76,6.0,3.0,4.0,6.0,False
4,1.20,2490,4.80,75,4.0,10.0,3.0,7.0,False
5,0.60,12143,4.80,69,1.0,2.0,3.0,8.0,False
6,1.30,5257,4.75,83,5.0,7.0,2.0,2.0,False
7,2.00,7590,4.90,79,7.0,4.0,5.0,4.0,False
8,2.50,5690,4.90,83,9.0,6.0,5.0,2.0,False
9,4.00,12790,4.90,78,10.0,1.0,5.0,5.0,False


In [41]:
def ideal(df_raw, param_dict):
    df_proc = df_raw.copy()[list(param_dict.keys())]
    value_cols = []
    for param in param_dict.keys():
        df_proc[param+'_rank'] = df_proc[param].rank(ascending=param_dict[param], method='dense')
        df_proc[param+'_value']= (df_proc[param+'_rank'].max()-df_proc[param+'_rank'])**2
        value_cols.append(param+'_value')
    df_proc['ideal'] = 0
    for col in value_cols:
        df_proc['ideal'] += df_proc[col]
    return df_proc

In [42]:
ideal(df, {'volume': True,
           'price': False})

,volume,price,volume_rank,volume_value,price_rank,price_value,ideal
0,0.80,2776,3.0,49.0,9.0,1.0,50.0
1,0.65,6394,2.0,64.0,5.0,25.0,89.0
2,2.40,4468,8.0,4.0,8.0,4.0,8.0
3,1.50,8090,6.0,16.0,3.0,49.0,65.0
4,1.20,2490,4.0,36.0,10.0,0.0,36.0
5,0.60,12143,1.0,81.0,2.0,64.0,145.0
6,1.30,5257,5.0,25.0,7.0,9.0,34.0
7,2.00,7590,7.0,9.0,4.0,36.0,45.0
8,2.50,5690,9.0,1.0,6.0,16.0,17.0
9,4.00,12790,10.0,0.0,1.0,81.0,81.0


In [43]:
df.head(5)

,link,volume,price,rating,noise
0,ozon.ru/product/293783766,0.80,2776,4.80,68
1,ozon.ru/product/151517600,0.65,6394,4.70,80
2,ozon.ru/product/8751202,2.40,4468,4.90,84
3,ozon.ru/product/147927590,1.50,8090,4.85,76
4,ozon.ru/product/278676526,1.20,2490,4.80,75


In [52]:
ideal(df, {'volume': True,
           'price': False,
           'rating': True,
           'noise': False}).sort_values('ideal')

,volume,price,rating,noise,volume_rank,volume_value,price_rank,price_value,rating_rank,rating_value,noise_rank,noise_value,ideal
4,1.20,2490,4.80,75,4.0,36.0,10.0,0.0,3.0,4.0,7.0,4.0,44.0
0,0.80,2776,4.80,68,3.0,49.0,9.0,1.0,3.0,4.0,9.0,0.0,54.0
8,2.50,5690,4.90,83,9.0,1.0,6.0,16.0,5.0,0.0,2.0,49.0,66.0
7,2.00,7590,4.90,79,7.0,9.0,4.0,36.0,5.0,0.0,4.0,25.0,70.0
2,2.40,4468,4.90,84,8.0,4.0,8.0,4.0,5.0,0.0,1.0,64.0,72.0
3,1.50,8090,4.85,76,6.0,16.0,3.0,49.0,4.0,1.0,6.0,9.0,75.0
6,1.30,5257,4.75,83,5.0,25.0,7.0,9.0,2.0,9.0,2.0,49.0,92.0
9,4.00,12790,4.90,78,10.0,0.0,1.0,81.0,5.0,0.0,5.0,16.0,97.0
1,0.65,6394,4.70,80,2.0,64.0,5.0,25.0,1.0,16.0,3.0,36.0,141.0
5,0.60,12143,4.80,69,1.0,81.0,2.0,64.0,3.0,4.0,8.0,1.0,150.0


In [49]:
def linear_convolution(df_raw, param_dict, weights):
    df_proc = df_raw.copy()[list(param_dict.keys())]
    value_cols = []
    for param in param_dict.keys():
        df_proc[param+'_rank'] = df_proc[param].rank(ascending=param_dict[param], method='dense')
        df_proc[param+'_value']= df_proc[param+'_rank']*weights[param]
        value_cols.append(param+'_value')
    df_proc['convolution'] = 0
    for col in value_cols:
        df_proc['convolution'] += df_proc[col]
    return df_proc

In [54]:
linear_convolution(df,
                   {'volume': True,
                    'price': False,
                    'rating': True,
                    'noise': False},
                   {'volume': 0.2,
                    'price': 0.6,
                    'rating': 0.4,
                    'noise': 0.3}).sort_values('convolution')

,volume,price,rating,noise,volume_rank,volume_value,price_rank,price_value,rating_rank,rating_value,noise_rank,noise_value,convolution
1,0.65,6394,4.70,80,2.0,0.4,5.0,3.0,1.0,0.4,3.0,0.9,4.7
5,0.60,12143,4.80,69,1.0,0.2,2.0,1.2,3.0,1.2,8.0,2.4,5.0
9,4.00,12790,4.90,78,10.0,2.0,1.0,0.6,5.0,2.0,5.0,1.5,6.1
3,1.50,8090,4.85,76,6.0,1.2,3.0,1.8,4.0,1.6,6.0,1.8,6.4
6,1.30,5257,4.75,83,5.0,1.0,7.0,4.2,2.0,0.8,2.0,0.6,6.6
7,2.00,7590,4.90,79,7.0,1.4,4.0,2.4,5.0,2.0,4.0,1.2,7.0
8,2.50,5690,4.90,83,9.0,1.8,6.0,3.6,5.0,2.0,2.0,0.6,8.0
2,2.40,4468,4.90,84,8.0,1.6,8.0,4.8,5.0,2.0,1.0,0.3,8.7
0,0.80,2776,4.80,68,3.0,0.6,9.0,5.4,3.0,1.2,9.0,2.7,9.9
4,1.20,2490,4.80,75,4.0,0.8,10.0,6.0,3.0,1.2,7.0,2.1,10.1
